For this workshop, you need:

* An Azure Machine Learning workspace. 
* The Azure Machine Learning CLI v2 installed.

To install the CLI you can either,

Create a compute instance, which already has installed the latest AzureML CLI and is pre-configured for ML workflows.

Use the followings commands to install Azure ML CLI v2:

```bash
az extension add --name ml
az login --identity
```



# Model Training

## (Optional) 1. Create Managed Compute

A compute is a designated compute resource where you run your job or host your endpoint. Azure Machine learning supports the following types of compute:

- **Compute instance** - a fully configured and managed development environment in the cloud. You can use the instance as a training or inference compute for development and testing. It's similar to a virtual machine on the cloud.

- **Compute cluster** - a managed-compute infrastructure that allows you to easily create a cluster of CPU or GPU compute nodes in the cloud.

- **Inference cluster** - used to deploy trained machine learning models to Azure Kubernetes Service. You can create an Azure Kubernetes Service (AKS) cluster from your Azure ML workspace, or attach an existing AKS cluster.

- **Attached compute** - You can attach your own compute resources to your workspace and use them for training and inference.

You can create a compute using the Studio, the cli and the sdk.

<hr>

We can create a **compute instance** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_compute_instance.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


<hr>

We can create a **compute cluster** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_compute_cluster.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


Let's create a managed compute cluster for the training workload.

``` python
# Create train job compute cluster
!az ml compute create --file train/compute.yml
```

## 2. Register Data Asset

**Datastore** - Azure Machine Learning Datastores securely keep the connection information to your data storage on Azure, so you don't have to code it in your scripts.

An Azure Machine Learning datastore is a **reference** to an **existing** storage account on Azure. The benefits of creating and using a datastore are:
* A common and easy-to-use API to interact with different storage type. 
* Easier to discover useful datastores when working as a team.
* When using credential-based access (service principal/SAS/key), the connection information is secured so you don't have to code it in your scripts.

Supported Data Resources: 

* Azure Storage blob container
* Azure Storage file share
* Azure Data Lake Gen 1
* Azure Data Lake Gen 2


It is not a requirement to use Azure Machine Learning datastores - you can use storage URIs directly assuming you have access to the underlying data.

You can create a datastore using the Studio, the cli and the sdk.

<hr>

We can create a **datastore** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_datastore.png" width = "700px" alt="Create Datastore cli vs sdk">
</center>



**Data asset** - Create data assets in your workspace to share with team members, version, and track data lineage.

By creating a data asset, you create a reference to the data source location, along with a copy of its metadata. 

The benefits of creating data assets are:

* You can **share and reuse data** with other members of the team such that they do not need to remember file locations.
* You can **seamlessly access data** during model training (on any supported compute type) without worrying about connection strings or data paths.
* You can **version** the data.

<hr>

We can create a **data asset** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_data_asset.png" width = "700px" alt="Create Data Asset cli vs sdk">
</center>

In [1]:
# Register data asset 
!az ml data create --file train/data.yml

{
  "creation_context": {
    "created_at": "2022-12-17T21:49:48.920027+00:00",
    "created_by": "Louis Li (AI)",
    "created_by_type": "User",
    "last_modified_at": "2022-12-17T21:49:48.952502+00:00"
  },
  "description": "taxi dataset",
  "id": "/subscriptions/8480def5-8f7a-4285-99f7-295b61d7b22a/resourceGroups/mldemorg/providers/Microsoft.MachineLearningServices/workspaces/mldemo/data/taxi-data/versions/6",
  "name": "taxi-data",
  "path": "azureml://subscriptions/8480def5-8f7a-4285-99f7-295b61d7b22a/resourcegroups/mldemorg/workspaces/mldemo/datastores/workspaceblobstore/paths/LocalUpload/9292ec840b5d1db6306dba71da69ab7f/taxi-data.csv",
  "resourceGroup": "mldemorg",
  "tags": {},
  "type": "uri_file",
  "version": "6"
}


## 3. Register Train Environment

Azure Machine Learning environments define the execution environments for your **jobs** or **deployments** and encapsulate the dependencies for your code. 

Azure ML uses the environment specification to create the Docker container that your **training** or **scoring code** runs in on the specified compute target.

Create an environment from a
* conda specification
* Docker image
* Docker build context

There are two types of environments in Azure ML: **curated** and **custom environments**. Curated environments are predefined environments containing popular ML frameworks and tooling. Custom environments are user-defined.

<hr>

We can register an **environment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_environment.png" width = "700px" alt="Create Environment cli vs sdk">
</center>

In [3]:
# Register train environment 
!az ml environment create --file train/environment.yml

{
  "conda_file": {
    "channels": [
      "defaults",
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.7.5",
      "pip",
      {
        "pip": [
          "azureml-mlflow==1.38.0",
          "azure-ai-ml==1.0.0",
          "pyarrow==10.0.0",
          "ruamel.yaml==0.17.21",
          "scikit-learn==0.24.1",
          "pandas==1.2.1",
          "joblib==1.0.0",
          "matplotlib==3.3.3"
        ]
      }
    ]
  },
  "creation_context": {
    "created_at": "2022-12-17T21:50:31.521465+00:00",
    "created_by": "Louis Li (AI)",
    "created_by_type": "User",
    "last_modified_at": "2022-12-17T21:50:31.521465+00:00",
    "last_modified_by": "Louis Li (AI)",
    "last_modified_by_type": "User"
  },
  "description": "Environment created from a Docker image plus Conda environment to train taxi model.",
  "id": "azureml:/subscriptions/8480def5-8f7a-4285-99f7-295b61d7b22a/resourceGroups/mldemorg/providers/Microsoft.MachineLearningServices/workspaces/m

## 4. Create Pipeline Job

**AML Job**:

Azure ML provides several ways to train your models, from code-first solutions to low-code solutions:

* Azure ML supports script files in python, R, Java, Julia or C#. All you need to learn is YAML format and command lines to use Azure ML.

* Distributed Training: AML supports integrations with popular frameworks, PyTorch and TensorFlow. Both frameworks employ data parallelism & model parallelism for distributed training.

* Automated ML - Train models without extensive data science or programming knowledge.

* Designer - drag and drop web-based UI.

<hr>

We can submit a **job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_job.png" width = "700px" alt="Create Job cli vs sdk">
</center>

<br>
    
**AML Pipelines**:

An AML pipeline is an independently executable workflow of a complete machine learning task. It helps standardizing the best practices of producing a machine learning model: The core of a machine learning pipeline is to split a complete machine learning task into a multistep workflow. Each step is a manageable component that can be developed, optimized, configured, and automated individually. 

<hr>

We can submit a **pipeline job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_pipeline.png" width = "700px" alt="Create Pipeline cli vs sdk">
</center>

In [10]:
# Create pipeline job
!az ml job create --file train/pipeline.yml

Auto upgrade failed. name 'exit_code' is not defined
Traceback (most recent call last):
  File "/opt/az/extensions/ml/azext_mlv2/manual/vendored_curated_sdk/azure/ai/ml/entities/_util.py", line 143, in load_from_dict
    return schema(context=context).load(data, **kwargs)
  File "/opt/az/extensions/ml/marshmallow/schema.py", line 722, in load
    return self._do_load(
  File "/opt/az/extensions/ml/marshmallow/schema.py", line 909, in _do_load
    raise exc
marshmallow.exceptions.ValidationError: {'jobs': defaultdict(<class 'dict'>, {'prep_data': {'value': [{'code': [{'_schema': ['Value passed is not a data binding string: ../../../component/prep']}, {'_schema': ['Value passed is not a data binding string: ../../../component/prep']}, {'_schema': ['../../../component/prep is not a valid path']}, {'_schema': ['Not a valid URL.']}, {'_schema': ["In order to specify an existing codes, please provide either of the following prefixed with 'azureml:':\n1. The full ARM ID for the resource, e.g.

# Online Endpoint

Online endpoints are endpoints that are used for online (real-time) inferencing. They receive data from clients and can send responses back in real time.

An **endpoint** is an HTTPS endpoint that clients can call to receive the inferencing (scoring) output of a trained model. It provides:
* Authentication using "key & token" based auth
* SSL termination
* A stable scoring URI (endpoint-name.region.inference.ml.azure.com)

A **deployment** is a set of resources required for hosting the model that does the actual inferencing.
A single endpoint can contain multiple deployments.

Features of the managed online endpoint:

* **Test and deploy locally** for faster debugging
* Traffic to one deployment can also be **mirrored** (copied) to another deployment.
* **Application Insights integration**
* Security
* Authentication: Key and Azure ML Tokens
* Automatic Autoscaling
* Visual Studio Code debugging

**blue-green deployment**: An approach where a new version of a web service is introduced to production by deploying it to a small subset of users/requests before deploying it fully.

<center>
<img src="../../imgs/endpoint_concept.png" width = "500px" alt="Online Endpoint Concept cli vs sdk">
</center>

## 1. Create Online Endpoint

We can create an **online endpoint** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_online_endpoint.png" width = "700px" alt="Create Online Endpoint cli vs sdk">
</center>

In [ ]:
# create online endpoint
!az ml online-endpoint create --file deploy/online/online-endpoint.yml

## 2. Create Online Deployment

To create a deployment to online endpoint, you need to specify the following elements:

* Model files (or specify a registered model in your workspace)
* Scoring script - code needed to do scoring/inferencing
* Environment - a Docker image with Conda dependencies, or a dockerfile
* Compute instance & scale settings

Note that if you're deploying **MLFlow models**, there's no need to provide **a scoring script** and execution **environment**, as both are autogenerated.

We can create an **online deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_online_deployment.png" width = "700px" alt="Create Online Deployment cli vs sdk">
</center>

In [ ]:
# create online deployment
!az ml online-deployment create --file deploy/online/online-deployment.yml 

## 3. Allocate Traffic

In [ ]:
# allocate traffic
!az ml online-endpoint update --name taxi-online-endpoint --traffic blue=100

## 4. Invoke and Test Endpoint

We can invoke the **online deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/invoke_online_endpoint.png" width = "700px" alt="Invoke online endpoint cli vs sdk">
</center>

In [ ]:
# invoke and test endpoint
!az ml online-endpoint invoke --name taxi-online-endpoint --request-file ../../data/taxi-request.json

# Model Batch Endpoint

## 1. Create Batch Compute Cluster

In [ ]:
# create compute cluster to be used by batch cluster
!az ml compute create -n batch-cluster --type amlcompute --min-instances 0 --max-instances 3

## 2. Create Batch Endpoint

We can create the **batch endpoint** with cli v2 or sdk v2 using the following syntax:


<center>
<img src="../../imgs/create_batch_endpoint.png" width = "700px" alt="Create batch endpoint cli vs sdk">
</center>

In [ ]:
# create batch endpoint
!az ml batch-endpoint create --file deploy/batch/batch-endpoint.yml

## 3. Create Batch Deployment

We can create the **batch deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_batch_deployment.png" width = "700px" alt="Create batch deployment cli vs sdk">
</center>

Note that if you're deploying **MLFlow models**, there's no need to provide **a scoring script** and execution **environment**, as both are autogenerated.

In [ ]:
# create batch deployment
!az ml batch-deployment create --file deploy/batch/batch-deployment.yml --set-default

## 4. Invoke and Test Endpoint

We can invoke the **batch deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/invoke_batch_deployment.png" width = "700px" alt="Invoke batch deployment cli vs sdk">
</center>

In [ ]:
# invoke and test endpoint
!az ml batch-endpoint invoke --name taxi-batch-endpoint --input ../../data/taxi-batch.csv